In [2]:
import pandas as pd
import numpy as np
import os 
import sys
import glob 
import json
import importlib
import tqdm
import warnings

In [3]:
#Make sure to run this only once, to make lobbying-market the working directory
parent_dir_name=os.path.basename(os.path.dirname(os.getcwd()))
if parent_dir_name=='lobbying-market':
    os.chdir(os.path.pardir)
    print(os.getcwd())
    utils_path= os.path.abspath('utils')
    sys.path.append(utils_path)

import preprocessing as pp
importlib.reload(pp)

with open("LDA_data/issue_list.json", 'r') as f:
    issue_list = json.load(f)

issue_df=pd.read_csv('LDA_data/data_by_issue.csv')

c:\Users\nicoj\Documents\GitHub\lobbying-market


In [4]:
path_pattern='LDA_data/Filings_'
dirdict={str(i):path_pattern + str(i) for i in range(2013,2024)}
filepath_lists = {
    yearkey:
        [dirdict.get(yearkey)+'/'+filepath for filepath in os.listdir(dirdict.get(yearkey))] 
    for yearkey in dirdict
}

index_dict={
    'Q'+str(i+1): [6*i+j for j in range(6)]+[2*i+j for j in range(24,26)] for i in range(4)
}


file_list_dependency={
    yr_key : {
       qt_key:
           [filepath_lists[yr_key][i] for i in index_dict[qt_key]]
    for qt_key in index_dict
    }
    for yr_key in filepath_lists
}


In [5]:
try: preprocessed_file_dict.keys()
except: preprocessed_file_dict={}
dir_path='LDA_data/processed'
filelist= os.listdir(dir_path)
for yr in range(2013,2024):
    for j in range(4):
        key=str(yr)+str(j+1)
        i=(yr-2013)*4+j
        preprocessed_file_dict[key]=dir_path+'/'+filelist[i]  

In [ ]:
#df=pp.preprocess_data(pd.read_csv(file_list_dependency['2023']['Q4'][6]))
#This saves the preprocessed data into csv files
#It also saves the issue list
os.makedirs('LDA_data/processed', exist_ok=True) 
warnings.filterwarnings('ignore')
issue_set=set([])
Year_Keys=list(file_list_dependency.keys())
for yr_key in tqdm.tqdm(Year_Keys, desc = 'YEARS'):
    #print(yr_key)
    QT_Keys=list(file_list_dependency[yr_key].keys())
    for qt_key in tqdm.tqdm(QT_Keys ,desc = 'Quarters'):
        #print(qt_key)
        df=pp.preprocess_data(pd.read_csv(file_list_dependency[yr_key][qt_key][6]))
        filepath= 'LDA_data/processed/filings_'+yr_key+'_'+qt_key+'.csv'
        df.to_csv(filepath,index=False)
        issue_subset= set([col_name for col_name in df.columns if col_name.find('i_')>-1])
        issue_set= issue_set.union(issue_subset)
issue_list=list(issue_set)
issue_set.remove('i_NUMBER')
with open("LDA_data/issue_list.json", 'w') as f:
    json.dump(issue_list, f, indent=2)

""
issue
i_ALC
i_AVI
i_HCR
i_FIR
i_UTI
...
i_GOV
i_BAN
i_FUE


In [7]:
def add_data_by_issue(df,qt_key):
    df_to_load= pd.read_csv(preprocessed_file_dict[qt_key])
    #print(df_to_load.shape)
    for issue_key in df['issue']:
        #print(issue_key)
        #filter by issue==1
        filtered_df= df_to_load[df_to_load[issue_key] == 1]
        
                
        #count the rows in issue and add to dataframe
        issue_count= filtered_df.shape[0]
        col_name= 'count_'+qt_key
        df.loc[issue_df['issue']==issue_key,col_name]=issue_count
        #print(col_name+':'+str(issue_count))
        
        
        #count the number of na in both income and expenses then add to dataframe
        filter_na_income=filtered_df[filtered_df['income'].isna()]
        filter_na_expenses=filter_na_income[filter_na_income['expenses'].isna()]
        issue_count_na = filter_na_expenses.shape[0]
        col_name= 'nan_'+qt_key
        df.loc[issue_df['issue']==issue_key,col_name]=issue_count_na
        #print(col_name+':'+str(issue_count_na))
        
        #sum income then add to dataframe
        issue_income= sum(filtered_df['income'].dropna())
        col_name= 'income_'+qt_key
        df.loc[issue_df['issue']==issue_key,col_name]=issue_income
        #print(col_name +':'+str(issue_income))
        
        #sum expenses then add to dataframe
        issue_expenses= sum(filtered_df['expenses'].dropna())
        col_name= 'expenses_'+qt_key
        df.loc[issue_df['issue']==issue_key,col_name]=issue_expenses
        #print(col_name+':'+str(issue_expenses))
        
        #sum total then add to dataframe
        issue_total=issue_income+issue_expenses
        col_name= 'total_'+qt_key
        df.loc[issue_df['issue']==issue_key,col_name]=issue_total
        
        
        
    

In [12]:
warnings.filterwarnings('ignore')
issue_df= pd.DataFrame(issue_list,columns=['issue'])
issue_df.set_index('issue')
#add data to issue_df
for qt_key in tqdf(preprocessed_file_dict):
    add_data_by_issue(issue_df,qt_key)

100%|██████████| 44/44 [00:42<00:00,  1.03it/s]


In [13]:
issue_df.to_csv('LDA_data/data_by_issue.csv',index=False)